# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
EBOLA = DATA_FOLDER + "/ebola"
GUINEA = EBOLA + "/guinea_data"
LIBERIA = EBOLA + "/liberia_data"
SIERRA_LEONE = EBOLA + "/sl_data"

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
# Write your answer here
import pandas as pd
import numpy as np
import glob

def create_folder_data_frame(FOLDER):
    list_ = []
    for file_ in glob.glob(FOLDER + "/*.csv"):
        df = pd.read_csv(file_, index_col=None)
        list_.append(df)
    return pd.concat(list_)

def index_and_country(data_frame, country):
    data_frame["Country"] = country
    data_frame.set_index(['Date', 'Description', 'Country'], inplace=True)
    return data_frame
    
def preprocess_guinea(FOLDER):
    guinea_frame = create_folder_data_frame(FOLDER)
    guinea_frame.Date = pd.to_datetime(guinea_frame.Date)
    guinea_frame = index_and_country(guinea_frame, "Guinea")
    
    return guinea_frame
    
preprocessed_guinea = preprocess_guinea(GUINEA)

# Columns not towns: Date, Description Totals
preprocessed_guinea

In [ ]:
def preprocess_liberia(FOLDER):
    liberia_frame = create_folder_data_frame(FOLDER)
    liberia_frame.rename(columns={"Variable": "Description", "National": "Totals", 'Unnamed: 18': "Unknown"}, inplace=True)
    liberia_frame.Date = pd.to_datetime(liberia_frame.Date)
    liberia_frame = index_and_country(liberia_frame, "Liberia")
    
    return liberia_frame

preprocessed_liberia = preprocess_liberia(LIBERIA)
preprocessed_liberia

In [ ]:
def preprocess_sierra(FOLDER):
    sierra_frame = create_folder_data_frame(FOLDER)
    sierra_frame.rename(columns={"variable": "Description", "National": "Totals", 'Unnamed: 18': "Unknown", "date": "Date"}, inplace=True)
    sierra_frame.Date = pd.to_datetime(sierra_frame.Date)
    sierra_frame = index_and_country(sierra_frame, "Sierra Leone")
    
    return sierra_frame

#create_folder_data_frame(SIERRA_LEONE).columns
preprocessed_sierra = preprocess_sierra(SIERRA_LEONE)

In [ ]:
sierra_descriptions = set([b for a,b,c, in list(preprocessed_sierra.index.values)])
print("Sierra: {}\n".format(sierra_descriptions))

guinea_descriptions = set([b for a,b,c, in list(preprocessed_guinea.index.values)])
print("Guinea: {}\n".format(guinea_descriptions))

liberia_descriptions = set([b for a,b,c, in list(preprocessed_liberia.index.values)])
print("Liberia: {}\n".format(liberia_descriptions))
#sierra[~sierra['34 Military Hospital'].isnull()]
#preprocessed_sierra.iloc[0]


## 1.3 Liberia

- 'Total suspected cases’, 'Total confirmed cases', 'Total probable cases'  (octobre et ½ decembre NaN)
- ‘Cumulative confirmed, probable and suspected cases’ seulement 33 lignes (octobre novembre decembre présent)
- 'Cumulative (confirmed + probable + suspected)' : une seule date avec diplication d’index … (2014-10-04)
- 'Cumulative (confirmed + probable + suspects)' : 11 dates (octobre)
-  'Newly reported deaths' : toutes les dates mais : 12 NaN <br>
{Timestamp('2014-09-20 00:00:00'), Timestamp('2014-11-28 00:00:00'), Timestamp('2014-11-30 00:00:00'), Timestamp('2014-12-01 00:00:00'), Timestamp('2014-12-02 00:00:00'), Timestamp('2014-12-03 00:00:00'), Timestamp('2014-12-04 00:00:00'), Timestamp('2014-12-05 00:00:00'), Timestamp('2014-12-06 00:00:00'), Timestamp('2014-12-07 00:00:00'), Timestamp('2014-12-08 00:00:00'), Timestamp('2014-12-09 00:00:00')}
- 'Total death/s in confirmed, probable, suspected cases' : 76 lignes présentent (mais index en double pour le 10-04 …)  et deux dates NaN <br>
{Timestamp('2014-09-20 00:00:00'), Timestamp('2014-10-11 00:00:00')}



NaN Values des 6 descriptions de bases:
- New Suspected/Probable : 3 jours (alors que parfois il y a des 0)
- New Confirmed : tout le mois de Novembre ......
- Total death/s in confirmed cases /probable/suspected : octobre/novembre/decembre ... (3 mois sur 6)
- Newly reported deaths : 3 jours + decembre
- Total death/s in confirmed, probable, suspected cases : 4 jours + novembre/decembre
- Total confirmed cases : octobre, 1 jour en novembre, mi decembre

In [ ]:
# Les colonnes qui nous intérèssent le plus. Ce sont les plus complètent mais on va continuer à les
# analyser en faisant la df pour afiner les données.(voir markdown pour ecplication) 
interest_liberia = { 'New Case/s (Suspected)': 'new_suspected', 
        'New Case/s (Probable)': 'new_probable', 
        'New case/s (confirmed)': 'new_confirmed', 
        'Total death/s in suspected cases': 'death_suspected', 
        'Total death/s in probable cases': 'death_probable', 
        'Total death/s in confirmed cases': 'death_confirmed',
        'Total death/s in confirmed, probable, suspected cases': 'total_death'}

# We start by keeping only the interesting columns for the task
interest_liberia_indices = preprocessed_liberia.index.get_level_values('Description').isin(interest_liberia.keys())
liberia_df = preprocessed_liberia[interest_liberia_indices]

# Replacing all NaN values by 0 since we assumed that no value
# meant no new cases/new deaths on this day
# Transforming every dtype to integer
# Sorting on the date
liberia_df = liberia_df.fillna(0).astype(int).sort_index(0)

# Keeping only the interesting columns, namely one for the Total
# and one for the total of cities, to filter out unreliable data points
liberia_df['Cities_total'] = liberia_df.sum(1) - liberia_df.Totals
liberia_df = liberia_df[['Cities_total', 'Totals']]

# Now we want to veriy the rows for which the total of cities 
# diverges from the Totals column of at least 10%
liberia_df_limit = liberia_df[np.abs(liberia_df.Cities_total - liberia_df.Totals) > (0.1 * liberia_df.Totals)]

# Then we decide to remove the ones with cumulative (total) description,
# because the values are bigger thus the 10% divergences are more significant
liberia_df_total = liberia_df_limit.xs('Total death/s in confirmed, probable, suspected cases', level='Description')
total_date = liberia_df_total.index.get_level_values(0)

# We remove the rows with one zero and 10% of divergernce
# !!!!!!!!!!!!!!!!!!!!!!!
liberia_df_zero = liberia_df_limit[(liberia_df_limit.values == 0)]
zero_date = liberia_df_zero.index.get_level_values(0)

# Thus we drop these rows according to their date index
to_drop = total_date.append(zero_date).strftime('%Y-%m-%d')
liberia_df.drop(to_drop, level=0, inplace=True)

# We compute the mean between the total and the total of cities
# and we keep this final value. (!!!!!!!!!!!!!!)
liberia_df['Mean'] = liberia_df.mean(1)
liberia_df = liberia_df.Mean

# Converting the indices to columns, and readjusting the indices
liberia_df = liberia_df.unstack('Description', fill_value=0)
liberia_df.columns.rename('', inplace=True)

# changing the names of all indices to match other countries dataframes
liberia_df = liberia_df.rename(columns=interest_liberia)

# We sum the 'suspected, probable and confirmed' columns in order to get
# a more general and complete dataframe.
liberia_df['total_death_sum'] = liberia_df[['death_suspected','death_probable','death_confirmed']].sum(1)
liberia_df['new_cases_sum'] = liberia_df[['new_probable','new_suspected','new_confirmed']].sum(1)

# Moreover it allows us to compare the total death sum with the one present directly in the data
liberia_df_diff = liberia_df[liberia_df['total_death_sum'] != liberia_df['total_death']]
# ~~~~~~~~~~~~~~~~~~~~AFFICHER en HTML~~~~~~~~~~~~~~~~~~~~~
liberia_df_diff[['total_death','total_death_sum']]
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Markdown pour expliquer pour on garde 'Total death/s in confirmed, probable, suspected cases' à la place de la somme

# We keep our final columns
liberia_df = liberia_df[['total_death', 'new_cases_sum']]

# he data give us only the cumulated total for the deaths. So we computed the dayly numbers of deaths
liberia_df['total_death'] = liberia_df['total_death'] - liberia_df['total_death'][0]
liberia_df['total_death'][1:] = liberia_df['total_death'][1:].copy().as_matrix() - liberia_df['total_death'][:-1].copy().as_matrix()

# Moreover we have noticed that the last 6 rows of the new cases seem to be cumulative instead of daily
# that is why we process in the same way for these dates
liberia_df['new_cases_sum'][-6:] = liberia_df['new_cases_sum'][-6:] - liberia_df['new_cases_sum'][-6:][0]
liberia_df['new_cases_sum'][-6:][1:] = liberia_df['new_cases_sum'][-6:][1:].copy().as_matrix() - liberia_df['new_cases_sum'][-6:][:-1].copy().as_matrix()

#~~~~~~~~~~~~~~~~~~~ AFFICHER ~~~~~~~~~~~~~~~~~~~~~~~~~~~
print(liberia_df['total_death'].value_counts().sort_index())
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Markdown pour expliquer qu'on a eu des valeurs negatives qui sont apparus
# aux valeurs qui ne faisaient pas augmenter le total (fausse)
# et aussi il y a des extrems (les 4 valeurs au dessus de 1700)
#On met tout à zero

liberia_df[((liberia_df < 0) | (liberia_df > 1700))] = 0

#~~~~~~~~~~~~~~~~~AFFICHER LE FINAL~~~~~~~~~~~~~~~~~~~~~~
liberia_df

Guinea: Total new cases registered so far
        

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here
titanic_xls = pd.read_excel(DATA_FOLDER +'/titanic.xls')
titanic_xls